#  Intracranial Hemorrhage Detection

## Model 10: InceptionV3

#### Kristina Joos

---   

|                 	|                                                        	|
|:----------------:	|:-------------------------------------------------------:	|
| Model           	| InceptionV3                                    |
| Augmentation      | -                                                      |
| Windowing         | BSS Window                                                  	|
| Class Balancing 	| Oversampling 1:1,5 |
| Loss Function   	| Weighted LogLoss                                     	|
| Regularization  	| Drop out (0.2)                   |
| Epochs Run      	| 5                                                    	|
| Time Run (h)   	| 6                                         	|
|                 	|                                                        	|
| Test Sores      	| Loss:   0.0765                                     	|
| Validation      	| Loss:                                        	|
| Leader Board    	| Score: 0.39061 Rank: 403/1,345                               	|


## Imports

In [2]:
import numpy as np
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import collections
from tqdm import tqdm_notebook as tqdm
from datetime import datetime

from math import ceil, floor, log
import cv2

import tensorflow as tf

from tensorflow.keras import backend as K

import sys

# from keras_applications.resnet import ResNet50
from keras_applications.inception_v3 import InceptionV3




from skimage.transform import resize
from imgaug import augmenters as iaa
 
from sklearn.model_selection import train_test_split, StratifiedKFold, ShuffleSplit
from sklearn.metrics import confusion_matrix
import itertools

from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet_50
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg_16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Activation, concatenate, Dropout, MaxPooling2D, Conv2D, Flatten
from tensorflow.keras.initializers import glorot_normal, he_normal
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model, load_model, Sequential, load_model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.python.ops import array_ops

## Image Path

In [16]:
test_images_dir = '../data/input/rsna-intracranial-hemorrhage-detection/stage_2_test/'
train_images_dir = '../data/input/rsna-intracranial-hemorrhage-detection/stage_2_train/'

## Preprocessing

In [4]:
def window_with_correction(dcm, window_center, window_width):
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img

def window_without_correction(dcm, window_center, window_width):
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img

def bss_window(img, window):
    brain_img = window(img, 40, 80)
    subdural_img = window(img, 80, 200)
    soft_img = window(img, 40, 380)
    
    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = (soft_img - (-150)) / 380
    bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)

    return bss_img


def _read(path, desired_size):
    
    dcm = pydicom.dcmread(path)
    
    try:
        img = bss_window(dcm)
    except:
        img = np.zeros(desired_size)
    
    
    img = cv2.resize(img, desired_size[:2], interpolation=cv2.INTER_LINEAR)
    
    return img



class DataGenerator(keras.utils.Sequence):

    def __init__(self, list_IDs, labels=None, batch_size=1, img_size=(512, 512, 1), 
                 img_dir=train_images_dir, *args, **kwargs):

        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.indices) / self.batch_size))
    
    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indices]
        
        if self.labels is not None:
            X, Y = self.__data_generation(list_IDs_temp)
            return X, Y
        else:
            X = self.__data_generation(list_IDs_temp)
            return X

    def on_epoch_end(self):


        if self.labels is not None: 
            # Undersample
            keep_prob = self.labels.iloc[:, 0].map({0: 0.35, 1: 0.5})
            keep = (keep_prob > np.random.rand(len(keep_prob)))
            self.indices = np.arange(len(self.list_IDs))[keep]
            np.random.shuffle(self.indices)
        else:
            self.indices = np.arange(len(self.list_IDs))
            
    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.img_size))
        
        if self.labels is not None: # training phase
            Y = np.empty((self.batch_size, 6), dtype=np.float32)
        
            for i, ID in enumerate(list_IDs_temp):
                X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
                Y[i,] = self.labels.loc[ID].values
        
            return X, Y
        
        else: # test phase
            for i, ID in enumerate(list_IDs_temp):
                X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
            
            return X

## Custom Loss Function

In [5]:
# Code from here https://www.kaggle.com/akensert/inceptionv3-prev-resnet50-keras-baseline-model
def weighted_log_loss(y_true, y_pred):
    """
    Can be used as the loss function in model.compile()
    ---------------------------------------------------
    """
    
    class_weights = np.array([2., 1., 1., 1., 1., 1.])
    
    eps = K.epsilon()
    
    y_pred = K.clip(y_pred, eps, 1.0-eps)

    out = -(         y_true  * K.log(      y_pred) * class_weights
            + (1.0 - y_true) * K.log(1.0 - y_pred) * class_weights)
    
    return K.mean(out, axis=-1)


def _normalized_weighted_average(arr, weights=None):
    """
    A simple Keras implementation that mimics that of 
    numpy.average(), specifically for this competition
    """
    
    if weights is not None:
        scl = K.sum(weights)
        weights = K.expand_dims(weights, axis=1)
        return K.sum(K.dot(arr, weights), axis=1) / scl
    return K.mean(arr, axis=1)


In [6]:
# Code from here https://www.kaggle.com/akensert/inceptionv3-prev-resnet50-keras-baseline-model
def weighted_loss(y_true, y_pred):
    
    class_weights = K.variable([2., 1., 1., 1., 1., 1.])
    
    eps = K.epsilon()
    
    y_pred = K.clip(y_pred, eps, 1.0-eps)

    loss = -(        y_true  * K.log(      y_pred)
            + (1.0 - y_true) * K.log(1.0 - y_pred))
    
    loss_samples = _normalized_weighted_average(loss, class_weights)
    
    return K.mean(loss_samples)


In [7]:
# code from here https://www.kaggle.com/akensert/inceptionv3-prev-resnet50-keras-baseline-model
def weighted_log_loss_metric(trues, preds):
    
    class_weights = [2., 1., 1., 1., 1., 1.]
    
    epsilon = 1e-7
    
    preds = np.clip(preds, epsilon, 1-epsilon)
    loss = trues * np.log(preds) + (1 - trues) * np.log(1 - preds)
    loss_samples = np.average(loss, axis=1, weights=class_weights)

    return - loss_samples.mean()


In [17]:
# code from here https://www.kaggle.com/akensert/inceptionv3-prev-resnet50-keras-baseline-model
class PredictionCheckpoint(keras.callbacks.Callback):
    
    def __init__(self, test_df, valid_df, 
                 test_images_dir=test_images_dir, 
                 valid_images_dir=train_images_dir, 
                 batch_size=32, input_size=(224, 224, 3)):
        
        self.test_df = test_df
        self.valid_df = valid_df
        self.test_images_dir = test_images_dir
        self.valid_images_dir = valid_images_dir
        self.batch_size = batch_size
        self.input_size = input_size
        
    def on_train_begin(self, logs={}):
        self.test_predictions = []
        self.valid_predictions = []
        
    def on_epoch_end(self,batch, logs={}):
        self.test_predictions.append(
            self.model.predict_generator(
                DataGenerator(self.test_df.index, None, self.batch_size, self.input_size, self.test_images_dir), verbose=2)[:len(self.test_df)])
        

## Make Model

Code adapted from: https://www.kaggle.com/akensert/inceptionv3-prev-resnet50-keras-baseline-model

In [19]:
class MakeModel:
    
    def __init__(self, base_model, input_dims, batch_size=5, num_epochs=4, learning_rate=1e-3, 
                 decay_rate=1.0, decay_steps=1, weights="imagenet", verbose=1):
        
        self.base_model = base_model
        self.input_dims = input_dims
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.decay_rate = decay_rate
        self.decay_steps = decay_steps
        self.weights = weights
        self.verbose = verbose
        self._build()
        
    def _build(self):
        
        
        base_model = self.base_model(
                            include_top=False,
                            weights=self.weights,
                            input_shape=self.input_dims,
                            backend = tensorflow.keras.backend, 
                            layers = tensorflow.keras.layers,
                            models = tensorflow.keras.models,
                            utils = tensorflow.keras.utils)
        
        x = GlobalAveragePooling2D(name='avg_pool')(base_model.output) 
        x = Dropout(0.2)(x)
        x = Dense(keras.backend.int_shape(x)[1], activation="relu", name="dense_hidden_1")(x)
        x = Dropout(0.1)(x)
        out = Dense(6, activation="sigmoid", name='output')(x)

        self.model = keras.Model(inputs=base_model.input, outputs=out)

        self.model.compile(loss="binary_crossentropy", optimizer=Adam(), metrics=['Accuracy', weighted_loss])
        
    def fit_and_predict(self, train_df, valid_df, test_df):

        pred_history = PredictionCheckpoint(test_df, valid_df, input_size=self.input_dims)
        # Adapt learning rate
        scheduler = keras.callbacks.LearningRateScheduler(lambda epoch: self.learning_rate * pow(self.decay_rate, floor(epoch / self.decay_steps)))

        self.model.fit_generator(
            DataGenerator(
                train_df.index, 
                train_df, 
                self.batch_size, 
                self.input_dims, 
                train_images_dir
            ),
            epochs=self.num_epochs,
            verbose=self.verbose,
            use_multiprocessing=True,
            workers=12,
            callbacks=[pred_history, scheduler]
        )

        return pred_history

    def save(self, path):
        self.model.save_weights(path)

    def load(self, path):
        self.model.load_weights(path)

## Meta Data


In [38]:
label = train_df.Label.values
train_df = train_df.ID.str.rsplit("_", n=1, expand=True)
train_df.drop_diblicates(inplace=True)
train_df.loc[:, "label"] = label
train_df = train_df.rename({0: "id", 1: "subtype"}, axis=1)
train_df = pd.pivot_table(train_df, index="id", columns="subtype", values="label")

label = test_df.Label.values
test_df = test_df.ID.str.rsplit("_", n=1, expand=True)
test_df.loc[:, "label"] = label
test_df = test_df.rename({0: "id", 1: "subtype"}, axis=1)
test_df = pd.pivot_table(test_df, index="id", columns="subtype", values="label")

In [39]:
test_df.head()

Label                                                          \
Diagnosis      any epidural intraparenchymal intraventricular subarachnoid   
Image                                                                        
ID_000000e27   0.5      0.5              0.5              0.5          0.5   
ID_000009146   0.5      0.5              0.5              0.5          0.5   
ID_00007b8cb   0.5      0.5              0.5              0.5          0.5   
ID_000134952   0.5      0.5              0.5              0.5          0.5   
ID_000176f2a   0.5      0.5              0.5              0.5          0.5   

                       
Diagnosis    subdural  
Image                  
ID_000000e27      0.5  
ID_000009146      0.5  
ID_00007b8cb      0.5  
ID_000134952      0.5  
ID_000176f2a      0.5

In [53]:
train_df.head()

Label                                                          \
Diagnosis      any epidural intraparenchymal intraventricular subarachnoid   
Image                                                                        
ID_000012eaf     0        0                0                0            0   
ID_000039fa0     0        0                0                0            0   
ID_00005679d     0        0                0                0            0   
ID_00008ce3c     0        0                0                0            0   
ID_0000950d7     0        0                0                0            0   

                       
Diagnosis    subdural  
Image                  
ID_000012eaf        0  
ID_000039fa0        0  
ID_00005679d        0  
ID_00008ce3c        0  
ID_0000950d7        0

## Train-Test_Split , Train and Predict

In [60]:
ss = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42).split(df.index)

train_idx, valid_idx = next(ss)

## Make Model
model = MakeModel(base_model=InceptionV3, input_dims=(256, 256, 3), batch_size=32, learning_rate=5e-4,
                    num_epochs=50, decay_rate=0.8, decay_steps=1, weights="imagenet", verbose=1)


history = model.fit_and_predict(df.iloc[train_idx], df.iloc[valid_idx], test_df)

Epoch 1/5
7879/7879 [==============================] - 4308s 547ms/step - loss: 0.1190 - weighted_loss: 0.1319
Epoch 2/5
7872/7872 [==============================] - 3197s 406ms/step - loss: 0.0939 - weighted_loss: 0.1037
Epoch 3/5
7869/7869 [==============================] - 3122s 397ms/step - loss: 0.0842 - weighted_loss: 0.0928
Epoch 4/5
7864/7864 [==============================] - 3158s 402ms/step - loss: 0.0764 - weighted_loss: 0.0841
Epoch 5/5
7874/7874 [==============================] - 3132s 398ms/step - loss: 0.0695 - weighted_loss: 0.0765


## Make Submission

In [61]:
test_df.iloc[:, :] = np.average(history.test_predictions, axis=0, weights=[0, 1, 2, 4, 6]) # let's do a weighted average for epochs (>1)

test_df = test_df.stack().reset_index()

test_df.insert(loc=0, column='ID', value=test_df['Image'].astype(str) + "_" + test_df['Diagnosis'])

test_df = test_df.drop(["Image", "Diagnosis"], axis=1)

test_df.to_csv('submission.csv', index=False)
